In [1]:
import pandas as pd
import pickle
import PySimpleGUI as gui
from pathlib import Path

In [ ]:
#Function to make prediction 
def check(mFile,buy_price,maint_price,doors,persons,lug_boot,safety):

    pickle_in = open(mFile,"rb")
    RF = pickle.load(pickle_in)
    
    data = {'buy_price' : buy_price,
            'maint_price' : maint_price,
            'doors' : doors,
            'persons' : persons,
            'lug_boot' : lug_boot,
            'safety' : safety}
 
    data = pd.DataFrame(data,index=[0])

    data['buy_price']=data['buy_price'].map({'High':0,'Low':1,'Medium':2,'Very High':3})
    data['maint_price']=data['maint_price'].map({'High':0,'Low':1,'Medium':2,'Very High':3})
    data['doors']=data['doors'].map({'2':0,'3':1,'4':2,'5+':3})
    data['persons']=data['persons'].map({'2':0,'4':1,'5+':2})
    data['lug_boot']=data['lug_boot'].map({'Big':0,'Medium':1,'Small':2})
    data['safety']=data['safety'].map({'High':0,'Low':1,'Medium':2})

    pred = RF.predict(data)
    
    df1 = pd.DataFrame(pred, columns=['Result'], index=['Result : '])
    df1['Result']=df1['Result'].map({0: "Acceptable",1: "Good",2: "Unacceptable",3: "Very good",})

    window.disappear()
    gui.popup_no_titlebar("Safety of the car : ", df1)
    window.reappear()
    
#Check if file is valid
def is_valid_path(filepath):
    if filepath and Path(filepath).exists():
        return True
    gui.popup_error("Filepath is not valid")
    return False

#GUI using Pysimplegui
layout = [
    [gui.Text("Model : Random Forest", pad=((82,0),(0,0)), justification="r")],
    [gui.Text("Model File :", s=15, justification="r"), gui.Input(key="-ModelFile-"),gui.FileBrowse(file_types=(("PICKLE file","*.pickle*"),))],
    [gui.Text("Car Price :", s=15, justification="r"), gui.Combo(["Low", "Medium", "High", "Very High"], key="-BPrice-")],
    [gui.Text("Maintenance Price :", s=15, justification="r"), gui.Combo(["Low", "Medium", "High", "Very High"], key="-MPrice-")],
    [gui.Text("Number of doors :", s=15, justification="r"), gui.Combo(["2", "3", "4","5+"], key="-Doors-")],
    [gui.Text("Number of seats :", s=15, justification="r"), gui.Combo(["2", "4", "5+"], key="-Persons-")],
    [gui.Text("Luggage size :", s=15, justification="r"), gui.Combo(["Small", "Medium", "Big"], key="-Luggage-")],
    [gui.Text("Safety :", s=15, justification="r"), gui.Combo(["Low", "Medium", "High"], key="-Safety-")],
    [gui.Exit(s=18, button_color="tomato", pad=((120,6),(10,2))),gui.Button("Check", s=18, pad=((6,0),(10,2)))],
    ]

window = gui.Window("Machine Learning GUI", layout)

while True:
    event, values = window.read()
    print(event,values)
    if event in (gui.WINDOW_CLOSED,"Exit") :
        break
    if event == "Check" :
        if (is_valid_path(values["-ModelFile-"])):
            check(
                mFile = values["-ModelFile-"],
                buy_price = values["-BPrice-"],
                maint_price = values["-MPrice-"],
                doors = values["-Doors-"],
                persons = values["-Persons-"],
                lug_boot = values["-Luggage-"],
                safety = values["-Safety-"],
        )
window.close()